<a href="https://colab.research.google.com/github/satyajitghana/TSAI-DeepNLP-END2.0/blob/main/06_Encoder_Decoder/Tweets_Dataset.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [93]:
! pip install gdown==3.13.0 --quiet
! pip install spacy==3.0.6 --quiet
! pip install tweet-preprocessor --quiet
! pip install pytorch-lightning --quiet

     |████████████████████████████████| 808kB 5.2MB/s 
     |████████████████████████████████| 122kB 36.7MB/s 
     |████████████████████████████████| 10.6MB 32.6MB/s 
     |████████████████████████████████| 645kB 31.9MB/s 
     |████████████████████████████████| 829kB 38.5MB/s 
     |████████████████████████████████| 276kB 34.1MB/s 
     |████████████████████████████████| 1.3MB 30.5MB/s 
     |████████████████████████████████| 143kB 39.0MB/s 
     |████████████████████████████████| 296kB 29.6MB/s 
ERROR: tensorflow 2.5.0 has requirement tensorboard~=2.5, but you'll have tensorboard 2.4.1 which is incompatible.


In [ ]:
! python -m spacy download en_core_web_sm

In [2]:
! gdown https://drive.google.com/uc?id=1MMAXbWehsbs-tRpK2wxpF0UWrwC00hC5

Downloading...
From: https://drive.google.com/uc?id=1MMAXbWehsbs-tRpK2wxpF0UWrwC00hC5
To: /content/tweets.csv
100% 160k/160k [00:00<00:00, 5.07MB/s]


In [141]:
import pandas as pd
import preprocessor as p

from tqdm.auto import tqdm

tqdm.pandas()

/usr/local/lib/python3.7/dist-packages/tqdm/std.py:658: FutureWarning: The Panel class is removed from pandas. Accessing it from the top-level namespace will also be removed in the next version
  from pandas import Panel


In [142]:
dataset = pd.read_csv('tweets.csv')

In [143]:
dataset

,tweets,labels
0,Obama has called the GOP budget social Darwini...,1
1,"In his teen years, Obama has been known to use...",0
2,IPA Congratulates President Barack Obama for L...,0
3,RT @Professor_Why: #WhatsRomneyHiding - his co...,0
4,RT @wardollarshome: Obama has approved more ta...,1
...,...,...
1359,@liberalminds Its trending idiot.. Did you loo...,0
1360,RT @AstoldByBass: #KimKardashiansNextBoyfriend...,0
1361,RT @GatorNation41: gas was $1.92 when Obama to...,1
1362,"@xShwag haha i know im just so smart, i mean y...",1


In [144]:
dataset['labels'].value_counts()

0    931
1    352
2     81
Name: labels, dtype: int64

In [150]:
def clean_tweet(tweet):
    tweet = tweet.replace("RT", "")
    tweet = p.clean(tweet).strip()
    tweet = tweet.replace(":", "")
    return tweet

In [151]:
dataset['tweets_cleaned'] = dataset['tweets'].progress_apply(clean_tweet)
dataset = dataset[dataset['tweets_cleaned'].str.len() > 2]

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [152]:
dataset

,tweets,labels,tweets_cleaned
0,Obama has called the GOP budget social Darwini...,1,Obama has called the GOP budget social Darwini...
1,"In his teen years, Obama has been known to use...",0,"In his teen years, Obama has been known to use..."
2,IPA Congratulates President Barack Obama for L...,0,IPA Congratulates President Barack Obama for L...
3,RT @Professor_Why: #WhatsRomneyHiding - his co...,0,- his connection to supporters of Critical Ra...
4,RT @wardollarshome: Obama has approved more ta...,1,Obama has approved more targeted assassinatio...
...,...,...,...
1359,@liberalminds Its trending idiot.. Did you loo...,0,Its trending idiot.. Did you look at the tweet...
1360,RT @AstoldByBass: #KimKardashiansNextBoyfriend...,0,Barack Obama
1361,RT @GatorNation41: gas was $1.92 when Obama to...,1,gas was $1.92 when Obama took office...I gues...
1362,"@xShwag haha i know im just so smart, i mean y...",1,"haha i know im just so smart, i mean you gotta..."


In [153]:
idx = 700
dataset['tweets'].iloc[idx], dataset['tweets_cleaned'].iloc[idx]

('Small Biz News Obama to sign bill to help companies go public http://t.co/w8RSgU6t',
 'Small Biz News Obama to sign bill to help companies go public')

In [154]:
dataset.to_csv('tweets_cleaned.csv')

In [155]:
import torch
import torchtext
import gdown

import pytorch_lightning as pl

from torch.utils.data import Dataset, DataLoader, random_split

from torchtext.data.utils import get_tokenizer
from torchtext.vocab import build_vocab_from_iterator

import torchtext.experimental.functional as text_f

from pathlib import Path
from functools import partial

import pandas as pd
import preprocessor as p

from typing import *

In [167]:
class TweetsDataset(Dataset):
    """
    """

    URL = 'https://drive.google.com/uc?id=1gCEb9iRVYet15O4Tqvrj9Fjq9lVbSEQg'
    OUTPUT = 'tweets_cleaned.csv'
 

    def __init__(self, root, vocab=None, vectors=None, text_transforms=None, label_transforms=None, ngrams=1):
        """Initiate text-classification dataset.
        Args:
            vocab: Vocabulary object used for dataset.
        """

        super(self.__class__, self).__init__()

        # if split not in ['train', 'test']:
        #     raise ValueError(f'split must be either ["train", "test"] unknown split {split}')

        if vocab and vectors:
            raise ValueError(f'both vocab and vectors cannot be provided')

        self.vocab = vocab
        self.vectors = vectors

        gdown.cached_download(self.URL, Path(root) / self.OUTPUT)

        self.generate_tweet_dataset(Path(root) / self.OUTPUT)

        tokenizer = get_tokenizer("spacy", language="en_core_web_sm")

        # the text transform can only work at the sentence level
        # the rest of tokenization and vocab is done by this class
        self.text_transform = text_f.sequential_transforms(tokenizer, text_f.ngrams_func(ngrams))

        self.vocab_transforms = text_f.sequential_transforms()
        self.vector_transforms = text_f.sequential_transforms()

        def build_vocab(data, transforms):
            def apply_transforms(data):
                for line in data:
                    yield transforms(line)
            return build_vocab_from_iterator(apply_transforms(data), len(data))

        if self.vectors:
            self.vector_transforms = text_f.sequential_transforms(
                partial(vectors.get_vecs_by_tokens, lower_case_backup=True)
            )
        elif self.vocab is None:
            # vocab is always built on the train dataset
            self.vocab = build_vocab(self.dataset["tweets_cleaned"], self.text_transform)
    
        if self.vocab:
            self.vocab_transforms = text_f.sequential_transforms(
                text_f.vocab_func(self.vocab), text_f.totensor(dtype=torch.long)
            )

        if text_transforms is not None:
            self.text_transform = text_f.sequential_transforms(
                self.text_transform, text_transforms, self.vocab_transforms, self.vector_transforms 
            )
        else:
            self.text_transform = text_f.sequential_transforms(
                self.text_transform, self.vocab_transforms, self.vector_transforms
            )

        self.label_transform = text_f.sequential_transforms(text_f.totensor(dtype=torch.long))

    def generate_tweet_dataset(self, dataset_file):
        self.dataset = pd.read_csv(dataset_file)

    def __getitem__(self, idx):
        text = self.text_transform(self.dataset['tweets_cleaned'].iloc[idx])
        label = self.label_transform(self.dataset['labels'].iloc[idx])
        return label, text, self.dataset['tweets'].iloc[idx]

    def __len__(self):
        return len(self.dataset)

    @staticmethod
    def get_labels():
        return ['zero', 'one', 'two']

    def get_vocab(self):
        return self.vocab

    def get_vectors(self):
        return self.vectors

    def collator_fn(self, raw_texts=False):
        def collate_fn(batch):
            
            labels, sequences, raw_texts = zip(*batch)

            labels = torch.stack(labels)

            lengths = torch.LongTensor([len(sequence) for sequence in sequences])

            # print('before padding: ', sequences[40])

            if not self.vectors:
                pad_idx = self.get_vocab()['<pad>']
                sequences = torch.nn.utils.rnn.pad_sequence(sequences, 
                                                            padding_value = pad_idx,
                                                            batch_first=True
                                                            )
            # print('after padding: ', sequences[40])
            if raw_texts:
                return labels, sequences, lengths, raw_texts
            else:
                return labels, sequences, lengths
        
        return collate_fn

In [157]:
vectors = torchtext.vocab.GloVe("6B")

In [159]:
dataset = TweetsDataset(root='.', vectors=vectors)

File exists: tweets_cleaned.csv


In [160]:
dataset[0]

(tensor(1),
 tensor([[ 0.1030,  0.4800, -0.5092,  ..., -0.0370, -1.3354,  0.9289],
         [-0.2106,  0.2268,  0.0637,  ..., -0.6105, -0.2069, -0.1013],
         [ 0.0712,  0.3448,  0.0420,  ..., -0.0353, -0.5097,  0.1368],
         ...,
         [-0.6792, -0.1221, -0.2291,  ...,  0.0274, -0.2866, -0.0303],
         [-0.1018, -1.1882, -0.4792,  ...,  0.1053,  0.9321, -0.1999],
         [-0.1256,  0.0136,  0.1031,  ..., -0.3422, -0.0224,  0.1368]]),
 'Obama has called the GOP budget social Darwinism. Nice try, but they believe in social creationism.')

In [161]:
dataset[3]

(tensor(0),
 tensor([[ 0.0000,  0.0000,  0.0000,  ...,  0.0000,  0.0000,  0.0000],
         [ 0.0406, -0.0284,  0.2501,  ..., -0.7256, -0.1930,  0.2275],
         [-0.2213,  0.4714, -0.1666,  ..., -0.4044, -0.6878,  0.0392],
         ...,
         [ 0.0084,  0.2877, -0.2347,  ..., -0.6641,  0.1030,  0.1219],
         [ 0.4704,  0.1487,  0.1824,  ...,  0.3937, -0.5171,  0.9472],
         [ 0.2336, -0.3992, -0.2368,  ..., -0.4330,  0.0723,  0.4403]]),
 'RT @Professor_Why: #WhatsRomneyHiding - his connection to supporters of Critical Race Theory.... Oh wait, that was Obama, not Romney...')

In [87]:
train_size = int(0.8 * len(dataset))
test_size = len(dataset) - train_size
train_dataset, test_dataset = torch.utils.data.random_split(dataset, [train_size, test_size])

In [90]:
train_size

1091

In [88]:
train_dataset[0]

(tensor(0),
 tensor([[ 0.0000,  0.0000,  0.0000,  ...,  0.0000,  0.0000,  0.0000],
         [ 0.6947,  0.2218,  0.1053,  ...,  0.3487, -0.6308,  0.3809],
         [-0.6233, -0.4243, -0.0353,  ..., -0.1361,  0.0987,  0.6090],
         ...,
         [ 0.3165, -0.0019, -0.3633,  ..., -0.3862,  0.0104,  0.6200],
         [ 0.2336, -0.3992, -0.2368,  ..., -0.4330,  0.0723,  0.4403],
         [ 0.0000,  0.0000,  0.0000,  ...,  0.0000,  0.0000,  0.0000]]),
 'RT @/SheCallsMeLarryThe "new" anti-Obama slur from the #RWNJ echo chamber is THUG? So we\'ve decided to just let ... http://t.co/lFYgXSNS')

In [89]:
test_dataset[0]

(tensor(0),
 tensor([[ 0.1191,  0.0495, -0.0815,  ...,  0.6728, -0.0497,  0.3084],
         [-0.0055,  0.0979,  0.3243,  ..., -0.2580, -0.5349, -0.3267],
         [ 0.1030,  0.4800, -0.5092,  ..., -0.0370, -1.3354,  0.9289],
         ...,
         [-0.0018, -0.3367, -0.1308,  ..., -0.9911, -0.4665,  0.1936],
         [ 0.2336, -0.3992, -0.2368,  ..., -0.4330,  0.0723,  0.4403],
         [ 0.0000,  0.0000,  0.0000,  ...,  0.0000,  0.0000,  0.0000]]),
 "Examiner Editorial: Obama's budget, not Ryan's, is 'antithetical' to history: Speaking at the American Society o... http://t.co/mNhPMR5U")

In [168]:
class TweetsDataModule(pl.LightningDataModule):
    """
    DataModule for Tweets, train, val, test splits and transforms
    """

    name = "tweets"

    def __init__(
        self,
        data_dir: str = '.',
        val_split: int = 1000,
        num_workers: int = 2,
        batch_size: int = 128,
        *args,
        **kwargs,
    ):
        """
        Args:
            data_dir: where to save/load the data
            val_split: how many of the training images to use for the validation split
            num_workers: how many workers to use for loading data
            normalize: If true applies image normalize
            batch_size: desired batch size.
        """
        super().__init__(*args, **kwargs)

        self.data_dir = data_dir
        self.val_split = val_split
        self.num_workers = num_workers
        self.batch_size = batch_size

        self.dataset_train = ...
        self.dataset_val = ...
        self.dataset_test = ...
        self.vectors = ...

        self.Dataset = TweetsDataset

    def prepare_data(self):
        """Saves IMDB files to `data_dir`"""
        self.Dataset(self.data_dir)

    def setup(self, stage: Optional[str] = None):
        """Split the train and valid dataset"""
        
        self.vectors = torchtext.vocab.GloVe("6B")

        self._dataset = self.Dataset(self.data_dir, vectors=self.vectors)

        train_size = len(self._dataset) - self.val_split
        test_size = self.val_split
        train_dataset, test_dataset = torch.utils.data.random_split(self._dataset, [train_size, test_size])
        
        self.dataset_train = train_dataset
        self.dataset_test = test_dataset

    def train_dataloader(self):
        loader = DataLoader(
            self.dataset_train,
            batch_size=self.batch_size,
            shuffle=True,
            num_workers=self.num_workers,
            pin_memory=True,
            collate_fn=self.collator_fn
        )
        return loader

    def val_dataloader(self):
        loader = DataLoader(
            self.dataset_test,
            batch_size=self.batch_size,
            shuffle=False,
            num_workers=self.num_workers,
            pin_memory=True,
            collate_fn=self.collator_fn
        )
        return loader

    def test_dataloader(self):
        loader = DataLoader(
            self.dataset_test,
            batch_size=self.batch_size,
            shuffle=False,
            num_workers=self.num_workers,
            pin_memory=True,
            collate_fn=self.collator_fn
        )
        return loader

    def get_vocab(self):
        return self._dataset.get_vocab()
    
    def get_vectors(self):
        return self._dataset.get_vectors()

    @property
    def default_transforms(self):
        train_transforms = {
            'text_transforms': text_f.sequential_transforms(
                random_deletion,
                random_swap
            ),
            'label_transforms': None
        }
        test_transforms = {
            'text_transforms': None,
            'label_transforms': None
        }

        return train_transforms, test_transforms

    @property
    def collator_fn(self):
        return self._dataset.collator_fn(raw_texts=True)

In [169]:
datamodule = TweetsDataModule()
datamodule.setup()

File exists: tweets_cleaned.csv


In [170]:
train_loader = datamodule.train_dataloader()
val_loader = datamodule.val_dataloader()
test_loader = datamodule.test_dataloader()

In [171]:
len(train_loader), len(val_loader), len(test_loader)

(3, 8, 8)

In [172]:
a, b, c, d = next(iter(train_loader))

Labels

In [173]:
a

tensor([0, 2, 0, 1, 0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0,
        1, 1, 0, 2, 0, 0, 0, 1, 0, 1, 2, 0, 2, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0,
        0, 0, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 0, 0, 1, 0, 2, 1,
        0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 1, 0, 0, 1, 0, 0, 1, 0,
        0, 0, 1, 0, 1, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 1, 0, 1, 0, 0, 0, 2, 0, 1,
        0, 1, 1, 2, 1, 0, 0, 0])

Embeddings

In [174]:
b

(tensor([[ 0.1030,  0.4800, -0.5092,  ..., -0.0370, -1.3354,  0.9289],
         [-0.4373,  0.0490,  0.2532,  ...,  0.4244,  0.0218, -0.0235],
         [-0.2852, -0.0139,  0.3161,  ..., -0.1086, -0.1435,  0.1485],
         ...,
         [-0.2576, -0.0571, -0.6719,  ..., -0.1604,  0.0467, -0.0706],
         [-0.2853,  0.1124, -0.1908,  ..., -0.7520,  0.5028,  0.1516],
         [-0.8840,  0.0581,  0.0185,  ..., -0.9500,  0.0468, -0.2897]]),
 tensor([[ 0.0000,  0.0000,  0.0000,  ...,  0.0000,  0.0000,  0.0000],
         [-0.1604, -0.1256, -0.0563,  ...,  0.3619,  0.0986, -0.1434],
         [-0.0903,  0.2437, -0.3083,  ..., -0.0562, -0.0163,  0.3354],
         ...,
         [ 0.1484, -0.0357,  0.2418,  ...,  0.0036,  0.2269,  0.4667],
         [ 0.0000,  0.0000,  0.0000,  ...,  0.0000,  0.0000,  0.0000],
         [ 0.0148, -0.4540, -0.2523,  ..., -0.0018,  0.1680,  0.4530]]),
 tensor([[-0.2194,  0.8138, -0.4369,  ...,  0.4701,  0.3635,  0.1106],
         [ 0.1002,  1.2208,  0.1626,  ..., -0

Lengths

In [175]:
c

tensor([11, 19,  5,  9, 22, 21, 22, 15, 15, 16, 12, 24, 29, 12, 12, 27, 22,  6,
        20, 13,  7, 19,  6, 23, 12, 19,  6, 27, 17, 22, 20, 16, 37, 13, 14, 23,
        19, 19, 11, 24, 12, 28, 24, 15, 10, 20,  9, 12, 16, 21, 20, 20, 22, 14,
         6,  8, 25, 20, 37, 10,  7, 23, 28, 14, 11, 32,  8,  9, 13, 21, 18, 17,
        20,  8, 30, 27, 15, 26,  6,  6,  5, 27,  9,  9,  9,  4, 12, 13, 16, 21,
        13, 21, 15, 16, 29,  3, 30, 20, 27, 21, 13, 37, 37, 10, 11, 29,  9, 31,
        37, 10, 23, 19,  6,  7, 37, 14, 25, 26,  7, 38, 37, 27, 18, 17, 27, 25,
        28,  4])

Actual Tweets

In [176]:
d

('Obama Reveals How to Get National Unemployment Rate to 6% http://t.co/qHrUeEoO',
 'RT @GlobalGrind: Picture Perfect! Jay, Barack, Bey & Michelle are perfectly posed..photos &gt;&gt; http://t.co/XpXJJp9c',
 '#obama samsung galaxy i5800 android 2.2 http://t.co/bgk2uOdY',
 'Obama signs JOBS Act to boost startup firms - http://t.co/TFuPTCwT',
 'RT @jodikyman: RT @American1st UNELECTED OBAMA JUSTICE DEPT IS TRYIN 2 BULLY ELECTED SHERIFF JOE ARPAIO OF MARICOPA COUNTY? VIOLATING UR OWN RULE?Mr Obama',
 "RT @CBrielle5: Obama a goon lol RT @BarackObama: So what's Romney hiding? Tweet @MittRomney to demand he release his tax returns. #WhatsRomneyHiding",
 'RT @markknoller: Obama Campaign says Romney should make public 23 years of tax returns so Americans can assess any conflicts of interest.',
 "RT @Ooh_ShePretty: - I'm Riding With Him Like Obama , He Don't Gimme Drama",
 'In his teen years, Obama has been known to use marijuana and cocaine.',
 '"who was the woman who worked for Hillarys presi